<a href="https://colab.research.google.com/github/ele9996/keyword-spotting/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import os
from zipfile import ZipFile

In [4]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [6]:
uri = "http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip"
zip_path = keras.utils.get_file(origin=uri, fname="mini_speech_commands.zip")
zip_file = ZipFile(zip_path)
zip_file.extractall()


182083584/182082353 [==============================] - 1s 0us/step
